# Final Project - Beat the Bookies

## Day One - Initial Data Exploration

The goal of today is:

- Create repositories
- Add collaborators
- Add notebooks to the project
- Confirm primary data source
- Build a dataframe that includes all the relevant and desired data from our model (check google sheets for the columns). 


Using the dataset downloaded from kaggle (https://www.kaggle.com/hugomathien/soccer?) - exploring the data available for one season (2015/16) to combine the data into our own dataframe that includes the relevant information that we need for our models.

Colummns for the dataset:

- Home Team Rank
- Home Team Total Wins
- Home Team win
- Draw
- Away Team Win
- Away Team Rank

To add columns including possesion, reds, yellows, corners, shots at goal, shots on target, goal percentage, throw ins (?), fouls, free kicks. Later on when we know that we can combine the core of the table easily for all seasons.

Target:
- Home Team win y/n

### imports 

In [ ]:
import pandas as pd
import numpy as np 
import sqlite3

### importing the data from sqlite

In [ ]:
# create the file path (include two beatthebookies)
path = '/Users/georgebrockman/code/georgebrockman/beatthebookies/beatthebookies/data/database.sqlite'
# create the connection
conn = sqlite3.connect(path)
# create the cursor
cursor = conn.cursor()

In [ ]:
# from the match table we want to select all matches played in the English Premier league for the 2015/16 season.
# Premier League - league_id = 1729
# order by stages (Game Weeks)
# need quotes around '2015/2016'
result = cursor.execute(""" SELECT * FROM "Match" m 
                        WHERE league_id = 1729 
                        AND season = '2015/2016' 
                        ORDER BY stage ASC""")

In [ ]:
row = result.fetchall()
row[0] # seems to be a lot of null values for shots, s

In [ ]:
# create a DataFrame from the SQL query
matches_df = pd.read_sql(""" SELECT * FROM "Match" m 
                        WHERE league_id = 1729 
                        AND season = '2015/2016' 
                        ORDER BY stage ASC""", conn)

In [ ]:
# inspect the DataFrame
matches_df.columns.values

In [ ]:
# we can remove all the ones not needed for home-away goal data
simple_matches = matches_df.drop(['home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'away_player_Y8', 'away_player_Y9', 'away_player_Y10',
       'away_player_Y11', 'home_player_1', 'home_player_2',
       'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',
       'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1',
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11', 'goal', 'shoton', 'shotoff',
       'foulcommit', 'card', 'cross', 'corner', 'possession', 'B365H',
       'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH',
       'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH',
       'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH',
       'BSD', 'BSA'], axis=1)

In [ ]:
simple_matches.head(5)

In [ ]:
simple_matches['home_goals']= simple_matches.groupby('home_team_api_id')['home_team_goal'].cumsum().fillna(0)

In [ ]:
simple_matches['away_goals']= simple_matches.groupby('away_team_api_id')['away_team_goal'].cumsum().fillna(0)

In [ ]:
simple_matches

In [ ]:
simple_matches.info()

In [ ]:
simple_matches['home_wins'] = simple_matches.home_team_goal > simple_matches.away_team_goal
simple_matches['draws'] = simple_matches.home_team_goal == simple_matches.away_team_goal
simple_matches['away_wins'] = simple_matches.home_team_goal < simple_matches.away_team_goal
simple_matches

In [ ]:
# convert booleans into binary numbers 
simple_matches['home_wins'] = simple_matches['home_wins'].astype(int)
simple_matches['draws'] = simple_matches['draws'].astype(int)
simple_matches['away_wins'] = simple_matches['away_wins'].astype(int)


In [ ]:
simple_matches.head()

In [ ]:
# create cumulative home, draw and away wins grouping by each team id

simple_matches['cumulative_home_wins_home'] = simple_matches.groupby('home_team_api_id')['home_wins'].cumsum().fillna(0)
simple_matches['cumulative_away_wins_home'] = simple_matches.groupby('home_team_api_id')['home_wins'].cumsum().fillna(0)
simple_matches['cumulative_home_wins_away'] = simple_matches.groupby('away_team_api_id')['away_wins'].cumsum().fillna(0)
# simple_matches['cumulative_draws'] = simple_matches.groupby('home_team_api_id')['draws'].cumsum().fillna(0)
simple_matches['cumulative_away_wins_away'] = simple_matches.groupby('away_team_api_id')['away_wins'].cumsum().fillna(0)


In [ ]:
simple_matches.tail(20)

## Day two - Comparing bets (2015/2016)

The goal of today is to find a baseline betting profit and loss for our model to compare with.

From the data set we are going to look at the odds offered by the betting firm Ladbrokes on home win, away win and draw.

- Placing a ten pound bet on the favourite (team with lowest odds).
- Placing a ten pound bet on the Home team every time.
- Placing a ten pound bet on a draw every match.
- placing a ten pound bet on the underdog (team with highest odds).
- placing a ten pound bet on the away team.

Maximum loss is the ten pound stake - so after 380 matches worst case scenario is **£3800 down**. 


### Import the betting data

In [ ]:
import pandas as pd
import numpy as np 
import sqlite3

In [ ]:
# create the file path (include two beatthebookies)
path = '/Users/georgebrockman/code/georgebrockman/beatthebookies/beatthebookies/data/database.sqlite'
# create the connection
conn = sqlite3.connect(path)
# create the cursor
cursor = conn.cursor()

In [ ]:
# use joining tables to keep the team names for better clarity
matches = pd.read_sql("""SELECT m.id, 
                        m.season, m.stage, m.date, 
                        ht.team_long_name as home_team, at.team_long_name as away_team, m.home_team_goal, 
                        m.away_team_goal, LBH, LBD, LBA                                      
                        FROM Match as m
                        LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                        LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                        WHERE league_id = 1729 AND season = '2015/2016'
                        ;""", conn)

In [ ]:
# sort matches in to cronological order
matches.sort_values('date', inplace=True)

In [ ]:
# check correctly imported the required data
matches.head()

In [ ]:
# create booleans
matches['home_win'] = matches.home_team_goal > matches.away_team_goal
matches['draw'] = matches.home_team_goal == matches.away_team_goal
matches['away_win'] = matches.home_team_goal < matches.away_team_goal

In [ ]:
# convert booleans into binary numbers 
matches['home_win'] = matches['home_win'].astype(int)
matches['draw'] = matches['draw'].astype(int)
matches['away_win'] = matches['away_win'].astype(int)

### Profit from betting on favourite

In [ ]:
matches['fav_profit'] = -10

In [ ]:
matches.head()

In [ ]:
matches.loc[(matches[['LBH','LBD','LBA']].min(axis=1) == matches['LBH']) & (matches['home_win'] == 1), 'fav_profit'] = matches['LBH'] * 10
matches.loc[(matches[['LBH','LBD','LBA']].min(axis=1) == matches['LBD']) & (matches['draw'] == 1), 'fav_profit'] = matches['LBD'] * 10
matches.loc[(matches[['LBH','LBD','LBA']].min(axis=1) == matches['LBA']) & (matches['away_win'] == 1), 'fav_profit'] = matches['LBA'] * 10


In [ ]:
matches

### Profit betting on Underdog

In [ ]:
matches['dog_profit'] = -10

In [ ]:
matches.loc[(matches[['LBH','LBD','LBA']].max(axis=1) == matches['LBH']) & (matches['home_win'] == 1), 'dog_profit'] = matches['LBH'] * 10
matches.loc[(matches[['LBH','LBD','LBA']].max(axis=1) == matches['LBD']) & (matches['draw'] == 1), 'dog_profit'] = matches['LBD'] * 10
matches.loc[(matches[['LBH','LBD','LBA']].max(axis=1) == matches['LBA']) & (matches['away_win'] == 1), 'dog_profit'] = matches['LBA'] * 10

In [ ]:
matches.head()

### Profit from Home / Draw / Away

In [ ]:
matches['home_profit'] = -10

In [ ]:
matches.loc[(matches['home_win'] == 1), 'home_profit'] = matches['LBH'] * 10 

In [ ]:
matches['draw_profit'] = -10

In [ ]:
matches.loc[(matches['draw'] == 1), 'draw_profit'] = matches['LBD'] * 10 

In [ ]:
matches['away_profit'] = -10

In [ ]:
matches.loc[(matches['away_win'] == 1), 'away_profit'] = matches['LBA'] * 10 

In [ ]:
matches.sort_values('date', inplace=True)

In [ ]:
matches

### Cumulative profit totals for each strategey

In [ ]:
# define a function that returns the maximum profit from each strategy
def c_profits(X):
    fav_profit_cum = X['fav_profit'].sum()
    dog_profit_cum = X['dog_profit'].sum()
    home_profit_cum = X['home_profit'].sum()
    draw_profit_cum = X['draw_profit'].sum()
    away_profit_cum = X['away_profit'].sum()
    return fav_profit_cum, dog_profit_cum, home_profit_cum,draw_profit_cum, away_profit_cum

In [ ]:
matches['away_profit'].cumsum().tail(1)

In [ ]:
c_profits(matches)

Very surpising that the underdog approach was the most profitable method of betting in this season but this was the year that Leicester City won the league. 

### Reorganise methods into functions to apply to all the data

In [ ]:
# psuedo code
# import the data using get_data():
# this returns a table with home_w, away_w and draw already added so need to add the boolean and conversion.
# first function favourites method
# second function is underdog 
# third is home team 
# fourth is draw
# fifth allways bet away. 

In [ ]:
def pick_the_fav(X, season, bet=10):
    """
    function returns a dataframe that includes the weekly profit/loss for season followng the favourites strategy
    """
    # set defaults profit equal to bet size
    X['fav_profit'] = -bet
    # update profit column
    X.loc[(matches[['LBH','LBD','LBA']].min(axis=1) == X['LBH']) & (X['home_w'] == 1), 'fav_profit'] = X['LBH'] * 10
    X.loc[(matches[['LBH','LBD','LBA']].min(axis=1) == X['LBD']) & (X['draw'] == 1), 'fav_profit'] = X['LBD'] * 10
    X.loc[(matches[['LBH','LBD','LBA']].min(axis=1) == X['LBA']) & (X['away_w'] == 1), 'fav_profit'] = X['LBA'] * 10
    
    return X

In [ ]:
def pick_the_dog(X, season, bet=10):
    """
    function returns a dataframe that includes the weekly profit/loss for season followng the underdog strategy
    """
    # set defaults profit equal to bet size
    X['dog_profit'] = -bet
    # update profit column 
    X.loc[(matches[['LBH','LBD','LBA']].max(axis=1) == X['LBH']) & (X['home_w'] == 1), 'dog_profit'] = X['LBH'] * 10
    X.loc[(matches[['LBH','LBD','LBA']].max(axis=1) == X['LBD']) & (X['draw'] == 1), 'dog_profit'] = X['LBD'] * 10
    X.loc[(matches[['LBH','LBD','LBA']].max(axis=1) == X['LBA']) & (X['away_w'] == 1), 'dog_profit'] = X['LBA'] * 10
    
    return X

In [ ]:
def always_home(X, season, bet=10):
    """
    function returns a dataframe that includes the weekly profit/loss for season following the home team strategy
    """
    X['home_profit'] = -bet
    X.loc[(X['home_w'] == 1), 'home_profit'] = X['LBH'] * 10 
    
    return X

In [ ]:
def always_draw(X, season, bet=10):
    """
    function returns a dataframe that includes the weekly profit/loss for season following the home team strategy
    """
    X['draw_profit'] = -bet
    X.loc[(X['draw'] == 1), 'draw_profit'] = X['LBH'] * 10 
    
    return X

In [ ]:
def always_away(X, season, bet=10):
    """
    function returns a dataframe that includes the weekly profit/loss for season following the home team strategy
    """
    X['away_profit'] = -bet
    X.loc[(X['away_w'] == 1), 'away_profit'] = X['LBH'] * 10 
    
    return X

### test functions

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from beatthebookies.data import get_betting_data, get_data

In [ ]:
def get_betting_data(season='2015/2016', league=1729, local=False, optimize=False, **kwargs):
    path = "../data/"
    database = path + 'database.sqlite'
    conn = sqlite3.connect(database)

    df = pd.read_sql("""SELECT m.id,
                            m.season, m.stage, m.date,
                            ht.team_long_name as home_team, at.team_long_name as away_team, m.home_team_goal,
                            m.away_team_goal, m.LBH, m.LBD, m.LBA 
                            FROM Match as m
                            LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                            LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                            WHERE league_id = :league AND season = :season
                            ;""", conn, params={"league":league, "season":season})
    # add win columns
    df['home_w'] = 0
    df['away_w'] = 0
    df['draw'] = 0
    # set winner
    df.loc[df['home_team_goal'] > df['away_team_goal'], 'home_w'] = 1
    df.loc[df['home_team_goal'] < df['away_team_goal'], 'away_w'] = 1
    df.loc[df['home_team_goal'] == df['away_team_goal'], 'draw'] = 1
    # sort into order
    df.sort_values('date', inplace=True)

    return df

In [ ]:
betting = get_data()

In [ ]:
betting.info()

In [ ]:
def simple_betting_profits(df, bet=10):
    """
    function returns the cumulative profits/loss from a season of following a consistent, simple betting strategy.
    """
    # set defaults profit equal to bet size
    # create new column for fav strategy
    df['fav_profit'] = -bet
    # update profit column
    df.loc[(df[['LBH','LBD','LBA']].min(axis=1) == df['LBH']) & (df['home_w'] == 1), 'fav_profit'] = (df['LBH'] * bet) - bet
    df.loc[(df[['LBH','LBD','LBA']].min(axis=1) == df['LBD']) & (df['draw'] == 1), 'fav_profit'] = (df['LBD'] * bet) - bet
    df.loc[(df[['LBH','LBD','LBA']].min(axis=1) == df['LBA']) & (df['away_w'] == 1), 'fav_profit'] = (df['LBA'] * bet) - bet

    # set defaults profit equal to bet size
    # create new column for underdog strategy
    df['dog_profit'] = -bet
    # update profit column
    df.loc[(df[['LBH','LBD','LBA']].max(axis=1) == df['LBH']) & (df['home_w'] == 1), 'dog_profit'] = (df['LBH'] * bet) - bet
    df.loc[(df[['LBH','LBD','LBA']].max(axis=1) == df['LBD']) & (df['draw'] == 1), 'dog_profit'] = (df['LBD'] * bet) - bet
    df.loc[(df[['LBH','LBD','LBA']].max(axis=1) == df['LBA']) & (df['away_w'] == 1), 'dog_profit'] = (df['LBA'] * bet) - bet

    # create new column for home team method
    df['home_profit'] = -bet
    df.loc[(df['home_w'] == 1), 'home_profit'] = (df['LBH'] * bet) - bet
    # create new column for draw tactic
    df['draw_profit'] = -bet
    df.loc[(df['draw'] == 1), 'draw_profit'] = (df['LBD'] * bet) - bet
    # create new column for betting on the away team
    df['away_profit'] = -bet
    df.loc[(df['away_w'] == 1), 'away_profit'] = (df['LBA'] * bet) - bet

    fav_profit_total = df['fav_profit'].sum()
    dog_profit_total = df['dog_profit'].sum()
    home_profit_total = df['home_profit'].sum()
    draw_profit_total = df['draw_profit'].sum()
    away_profit_total = df['away_profit'].sum()

    return fav_profit_total, dog_profit_total, home_profit_total, draw_profit_total, away_profit_total


In [ ]:
simple_betting_profits(betting)

## Day 3 - Ternery outputs and introducing additional Data

In [1]:
import pandas as pd
import numpy as np 
import sqlite3

In [2]:
# create the file path (include two beatthebookies)
path = '/Users/georgebrockman/code/georgebrockman/beatthebookies/beatthebookies/data/database.sqlite'
# create the connection
conn = sqlite3.connect(path)
# create the cursor
cursor = conn.cursor()

In [3]:
# use joining tables to keep the team names for better clarity
matches = pd.read_sql("""SELECT m.id, 
                        m.season, m.stage, m.date, 
                        ht.team_long_name as home_team, at.team_long_name as away_team, m.home_team_goal, 
                        m.away_team_goal, LBH, LBD, LBA, goal, shoton, shotoff, foulcommit, card, 
                        cross, corner, possession                                      
                        FROM Match as m
                        LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                        LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                        WHERE league_id = 1729 AND season = '2015/2016'
                        ;""", conn)

In [4]:
matches.head()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,LBH,LBD,LBA,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,4389,2015/2016,1,2015-08-09 00:00:00,Arsenal,West Ham United,0,2,1.29,5.5,12.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><stats...
1,4390,2015/2016,1,2015-08-08 00:00:00,Bournemouth,Aston Villa,0,1,2.05,3.3,4.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>58</comment><stats...
2,4391,2015/2016,1,2015-08-08 00:00:00,Chelsea,Swansea City,2,2,1.40,4.5,10.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><stats...
3,4392,2015/2016,1,2015-08-08 00:00:00,Everton,Watford,2,2,1.75,3.8,5.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><stats...
4,4393,2015/2016,1,2015-08-08 00:00:00,Leicester City,Sunderland,4,2,2.00,3.4,4.2,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><event_incident_typefk>743</ev...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>46</comment><stats...


From here we have concluded not to use the Kaggle dataset for the match stats instead we will compile a complete data frame from 10 seperate csv files downloaded from https://www.football-data.co.uk

In [5]:
# load all csv files as dataframes
df = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2008.csv")
df2 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2009.csv")
df3 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2010.csv")
df4 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2011.csv")
df5 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2012.csv")
df6 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2013.csv")
df7 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2014.csv")
df8 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2015.csv")
df9 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2016.csv")
df10 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2017.csv")
df11 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2018.csv")
df12 = pd.read_csv("/Users/georgebrockman/code/georgebrockman/Premier League CSV/premierleague2019.csv")

In [ ]:
# issues combining all at once so combine individually
combin_df = pd.concat([df,df2])
combin_df.reset_index(drop=True, inplace=True)

In [ ]:
# 2008-2010 dataset
temp = pd.concat([combin_df, df3])
temp.reset_index(drop=True, inplace=True)
temp

In [ ]:
# 2008-2011
tmp = pd.concat([temp, df4])
tmp.reset_index(drop=True, inplace=True)
tmp

In [ ]:
# create a merged csv for seasons 2008-2011 
tmp.to_csv( "combined_csv.csv", index=True, encoding='utf-8-sig')

In [ ]:
# 2012-2013 season merge
tmp1 = pd.concat([df5, df6])
tmp1.reset_index(drop=True, inplace=True)
tmp1

In [ ]:
# each csv has different values so need to drop columns that arn't in the kaggle dataset.
len(df.columns.values)

In [ ]:
df.head(20)

In [ ]:
df2.columns.values # GBH GBD GBA - GAMEBOOKERS

## convert csv files to relevant columns

In [18]:
df = df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [19]:
df2 = df2[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [20]:
df3 = df3[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [21]:
df4 = df4[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [22]:
df5 = df5[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [23]:
df6 = df6[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [24]:
df7 = df7[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [25]:
df8 = df8[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [26]:
df9 = df9[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [27]:
df10 = df10[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']]

In [28]:
df11 = df11[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR','WHH', 'WHD', 'WHA']]

In [29]:
df12 = df12[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF',
       'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA']] 

In [30]:
match_stats = pd.concat([df,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12])
match_stats.reset_index(drop=True, inplace=True)
match_stats

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,WHH,WHD,WHA
0,16/08/08,Arsenal,West Brom,1.0,0.0,H,1.0,0.0,H,H Webb,...,8.0,7.0,5.0,0.0,0.0,0.0,0.0,1.17,5.50,12.00
1,16/08/08,Bolton,Stoke,3.0,1.0,H,3.0,0.0,H,C Foy,...,12.0,4.0,3.0,1.0,2.0,0.0,0.0,1.80,3.10,4.00
2,16/08/08,Everton,Blackburn,2.0,3.0,A,1.0,1.0,D,A Marriner,...,9.0,3.0,5.0,2.0,2.0,0.0,0.0,1.95,3.10,3.50
3,16/08/08,Hull,Fulham,2.0,1.0,H,1.0,1.0,D,P Walton,...,9.0,5.0,6.0,3.0,0.0,0.0,0.0,2.50,2.90,2.62
4,16/08/08,Middlesbrough,Tottenham,2.0,1.0,H,0.0,0.0,D,M Atkinson,...,12.0,7.0,9.0,1.0,2.0,0.0,0.0,2.90,3.20,2.15
5,16/08/08,Sunderland,Liverpool,0.0,1.0,A,0.0,0.0,D,A Wiley,...,12.0,1.0,8.0,0.0,2.0,0.0,0.0,5.50,3.30,1.57
6,16/08/08,West Ham,Wigan,2.0,1.0,H,2.0,0.0,H,S Bennett,...,11.0,6.0,10.0,2.0,1.0,0.0,0.0,1.83,3.20,3.75
7,17/08/08,Aston Villa,Man City,4.0,2.0,H,0.0,0.0,D,P Dowd,...,11.0,7.0,8.0,0.0,1.0,0.0,0.0,1.91,3.20,3.50
8,17/08/08,Chelsea,Portsmouth,4.0,0.0,H,3.0,0.0,H,M Dean,...,6.0,8.0,3.0,0.0,1.0,0.0,0.0,1.30,4.20,8.50
9,17/08/08,Man United,Newcastle,1.0,1.0,D,1.0,1.0,D,M Riley,...,11.0,6.0,6.0,3.0,0.0,0.0,0.0,1.25,4.50,10.00


In [31]:
# create the data frame
match_stats.to_csv( "premierleaguematches.csv", index=True, encoding='utf-8-sig')